In [21]:
# Load the environment variables
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv("env_vars.env")) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# Hugging Face Hub LLM

In [22]:
from langchain import PromptTemplate
from langchain import HuggingFaceHub, LLMChain

template = """Question: {question} Answer: """
prompt = PromptTemplate(template=template, input_variables=['question'])
print(prompt)

# Initialize Hub LLM
hub_llm = HuggingFaceHub(repo_id='google/flan-t5-xl', model_kwargs={'temperature':1e-10})

# Create prompt template > LLM chain
llm_chain = LLMChain(prompt=prompt, llm=hub_llm)

input_variables=['question'] output_parser=None partial_variables={} template='Question: {question} Answer: ' template_format='f-string' validate_template=True


In [26]:
# Ask the question
question = "Who is the current chairman of the Federal Reserve?"
print(question)

# NOTE: This will likely time out!
# print(llm_chain.run(question))

Who is the current chairman of the Federal Reserve?


# OpenAI LLM

In [25]:
from langchain.llms import OpenAI

# Initialize the model
davinci = OpenAI(model_name="text-davinci-003")
llm_chain = LLMChain(prompt=prompt, llm=davinci)

In [27]:
# Ask the question
print(llm_chain.run(question))



The current chairman of the Federal Reserve is Jerome Powell.


# Prompt Templates

Two kinds of knowledge that can be "reasoned" by LLM:
- **Parametric knowlege** -- learned during training and stored in model parameters (weights)
- **Source knowledge** -- provided to the model at inference time as examples (via input prompt)

In [29]:
prompt = """The following is a conversation with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: User: What is the meaning of life? AI: """

openai.temperature = 1.0 # increase creativity/randomness of output
print(davinci(prompt))



The meaning of life is to find your own purpose and make the most of it. Life is what you make it!


In [31]:
# To help the model, we can give it a few examples of the type of answers we’d like
prompt = """The following are exerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: User: How are you? AI: I can't complain but sometimes I still do. 
User: What time is it? AI: It's time to get a watch. User: What is the meaning of life? AI: """

print(davinci(prompt))


The meaning of life is to live it to the fullest and enjoy every moment.


In [32]:
# FewShotPromptTemplate caters to source knowledge input
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, 
    {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

# create a example template
example_template = """ User: {query} AI: {answer} """

In [34]:
# create a prompt example from above template
example_prompt = PromptTemplate(input_variables=["query", "answer"], template=example_template)
print(example_prompt)

input_variables=['query', 'answer'] output_parser=None partial_variables={} template=' User: {query} AI: {answer} ' template_format='f-string' validate_template=True


In [35]:
# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: """

# and the suffix our user input and output indicator
suffix = """ User: {query} AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(examples=examples, example_prompt=example_prompt,
                                                 prefix=prefix, suffix=suffix, input_variables=["query"],
                                                 example_separator="\n\n")
print(few_shot_prompt_template)

input_variables=['query'] output_parser=None partial_variables={} examples=[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do."}, {'query': 'What time is it?', 'answer': "It's time to get a watch."}] example_selector=None example_prompt=PromptTemplate(input_variables=['query', 'answer'], output_parser=None, partial_variables={}, template=' User: {query} AI: {answer} ', template_format='f-string', validate_template=True) suffix=' User: {query} AI: ' example_separator='\n\n' prefix='The following are exerpts from conversations with an AI assistant. \nThe assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. \nHere are some examples: ' template_format='f-string' validate_template=True


In [36]:
query = "What is the meaning of life?"

print(few_shot_prompt_template.format(query=query))

The following are exerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: 

 User: How are you? AI: I can't complain but sometimes I still do. 

 User: What time is it? AI: It's time to get a watch. 

 User: What is the meaning of life? AI: 


In [37]:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

In [38]:
print(examples)

[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do."}, {'query': 'What time is it?', 'answer': "It's time to get a watch."}, {'query': 'What is the meaning of life?', 'answer': '42'}, {'query': 'What is the weather like today?', 'answer': 'Cloudy with a chance of memes.'}, {'query': 'What is your favorite movie?', 'answer': 'Terminator'}, {'query': 'Who is your best friend?', 'answer': 'Siri. We have spirited debates about the meaning of life.'}, {'query': 'What should I do today?', 'answer': 'Stop talking to chatbots on the internet and go outside.'}]


In [39]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(examples=examples,
                                              example_prompt=example_prompt,
                                              max_length=50 # this sets the max length that examples should be
                                             )
print(example_selector)

examples=[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do."}, {'query': 'What time is it?', 'answer': "It's time to get a watch."}, {'query': 'What is the meaning of life?', 'answer': '42'}, {'query': 'What is the weather like today?', 'answer': 'Cloudy with a chance of memes.'}, {'query': 'What is your favorite movie?', 'answer': 'Terminator'}, {'query': 'Who is your best friend?', 'answer': 'Siri. We have spirited debates about the meaning of life.'}, {'query': 'What should I do today?', 'answer': 'Stop talking to chatbots on the internet and go outside.'}] example_prompt=PromptTemplate(input_variables=['query', 'answer'], output_parser=None, partial_variables={}, template=' User: {query} AI: {answer} ', template_format='f-string', validate_template=True) get_text_length=<function _get_length_based at 0x11b59e3b0> max_length=50 example_text_lengths=[15, 14, 11, 16, 10, 19, 19]


In [40]:
# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector, # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)
print(dynamic_prompt_template)

input_variables=['query'] output_parser=None partial_variables={} examples=None example_selector=LengthBasedExampleSelector(examples=[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do."}, {'query': 'What time is it?', 'answer': "It's time to get a watch."}, {'query': 'What is the meaning of life?', 'answer': '42'}, {'query': 'What is the weather like today?', 'answer': 'Cloudy with a chance of memes.'}, {'query': 'What is your favorite movie?', 'answer': 'Terminator'}, {'query': 'Who is your best friend?', 'answer': 'Siri. We have spirited debates about the meaning of life.'}, {'query': 'What should I do today?', 'answer': 'Stop talking to chatbots on the internet and go outside.'}], example_prompt=PromptTemplate(input_variables=['query', 'answer'], output_parser=None, partial_variables={}, template=' User: {query} AI: {answer} ', template_format='f-string', validate_template=True), get_text_length=<function _get_length_based at 0x11b59e3b0>, max_length=50,

In [43]:
prompt = dynamic_prompt_template.format(query="How do birds fly?")
print(prompt)

The following are exerpts from conversations with an AI assistant. 
The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. 
Here are some examples: 
 User: How are you? AI: I can't complain but sometimes I still do. 
 User: What time is it? AI: It's time to get a watch. 
 User: What is the meaning of life? AI: 42 
 User: How do birds fly? AI: 


In [44]:
print(davinci(prompt))

 With a little help from their friends!


# Memory